# Dog Breed Classifier

Note: This notebook is written down for easier production deployment scenario. So I skip any data exploration step and model training step.

## System Diagram

![](system-diagram.png)

- Dog Detection Module: VGG-16 pretrained model
- Dog Breed Classifier: Big Transfer(BiT) pretrained model with fine-tuning

**Note: Both modules only use CPU for inference.**

## Import Libraries

In [1]:
!pip install torch torchvision

     |████████████████████████████████| 752.0 MB 5.3 kB/s  eta 0:00:01
     |████████████████████████████████| 6.6 MB 6.9 kB/s  eta 0:00:01


In [2]:
!git clone https://github.com/google-research/big_transfer.git
!pip install -r big_transfer/bit_pytorch/requirements.txt

Cloning into 'big_transfer'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 76 (delta 32), reused 58 (delta 15), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [5]:
import torch
import torchvision.models as models
import numpy as np
from glob import glob
from tqdm import tqdm

## Import Datasets

In [4]:
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip
!rm dogImages.zip

--2020-06-06 04:12:17--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.117.16
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.117.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip’

dogImages.zip       100%[===================>]   1.05G  21.0MB/s    in 55s     

2020-06-06 04:13:13 (19.7 MB/s) - ‘dogImages.zip’ saved [1132023110/1132023110]

Archive:  dogImages.zip
   creating: dogImages/
   creating: dogImages/test/
   creating: dogImages/train/
   creating: dogImages/valid/
   creating: dogImages/test/001.Affenpinscher/
  inflating: dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg  
  inflating: dogImages/test/001.Affenpinscher/Affenpinscher_00023.jpg  
  inflating: dogImages/test/001.Affenpinscher/Affenpinscher_00036.jpg  
  inflating: dogImages/test/001.Af

In [6]:
dog_files = np.array(glob('dogImages/*/*/*'))
print(f'There are {len(dog_files)} total dog images.')

There are 8351 total dog images.


## Dog Detector

In [7]:
dog_detector = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/jupyter/.cache/torch/checkpoints/vgg16-397923af.pth
